# External Data Download

This notebook acts as the first step in the data analysis pipeline. It takes UPRN (Unique Property Reference Number) information provided by WMFS incident reports and utilises it to download property-specific data. Additionally, it fetches weather data corresponding to the time of each incident. The aim is to enrich the incident dataset with additional contextual data for subsequent analysis. 

In [ ]:
# Importing the required packages
import pandas as pd
import requests
import time

In [ ]:
inc_info = pd.read_csv('../../data/final/inc_info.csv')

## UPRN Data

In [ ]:
api_key = '*********'

In [ ]:
uprns = list(inc_info['uprn'].unique())

In [ ]:
# Reading in any existing uprn data (to no repeat requests).
uprn_master = pd.read_csv('../../data/final/uprn/uprn_master.csv')
uprn_master = uprn_master.drop(labels="Unnamed: 0", axis=1)

In [ ]:
# Starting a list for uprns resulting in errors, so they are not repeated.
error_uprns = []

In [ ]:
for i in uprns[:]:
    if uprn_master[uprn_master['uprn'] == i].shape[0] == 0 and i not in error_uprns:
        print("Gathering: " + str(i))
        
        try:
            # Request
            temp_response = requests.get('https://api.propertydata.co.uk/uprn?key=' + api_key + '&uprn=' + str(i))
            temp_response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes


            # Data to DataFrame
            temp_uprn_data = pd.DataFrame(temp_response.json())[['data']].T.reset_index()


            # Add UPRN to DataFrame
            temp_uprn_data['uprn'] = int(pd.DataFrame(temp_response.json())[['uprn']].iloc[0])

            # Concat to end of master

            uprn_master = pd.concat([uprn_master, temp_uprn_data])
            
        except requests.exceptions.RequestException as e:
            print("Error processing UPRN: " + str(i))
            print("Exception: " + str(e))
            error_uprns.append(i)  # Add the UPRN to the list of errors
            
        
        # Introduce a 2-second delay before the next iteration
        time.sleep(2)
        

    else:
        print("Already gathered: " + str(i))

In [ ]:
# Exporting UPRN data to csv file
uprn_master.to_csv('../data/final/uprn/uprn_master.csv')

## Weather Data

In [ ]:
# Merging the incident info data with uprn data and dropping NA values in the internal area column.
df = inc_info.merge(uprn_master, on="uprn", how="inner")
df = df.dropna(subset=["internalArea"])

In [ ]:
# Define a function to transform coordinates from EPSG:27700 to EPSG:4326
def transform_coordinates(row):
    transformer = Transformer.from_crs("EPSG:27700", "EPSG:4326")
    lon, lat = transformer.transform(row['location_x'], row['location_y'])
    return pd.Series({'longitude': lon, 'latitude': lat})


# Apply the transformation to the DataFrame
df[['longitude', 'latitude']] = df.apply(transform_coordinates, axis=1)


In [ ]:
# Convert the 'call_time' column to datetime format
df['call_time'] = pd.to_datetime(df['call_time'])

# Create an empty master DataFrame to store the extracted information
master_df = pd.DataFrame(columns=['vis_inc_num', 'date', 'mean_temperature', 'precipitation_sum', 'windspeed'])

# Loop through each incident in the DataFrame
for index, incident in df.iterrows():
    # Get the date of the incident
    date = incident['call_time'].date()
    
    # Fetch the API response for this incident
    api_response = requests.get("https://archive-api.open-meteo.com/v1/archive?latitude={}&longitude={}&start_date={}&end_date={}&daily=temperature_2m_mean,precipitation_sum,windspeed_10m_max&timezone=GMT".format(incident['latitude'], incident['longitude'], date - pd.Timedelta(days=7), date))
    
    # Parse the API response into a DataFrame
    api_response_df = pd.DataFrame(api_response.json()['daily'])
    
    # Extract the mean of the "temperature_2m_mean" column
    mean_temperature = api_response_df['temperature_2m_mean'].mean()

    # Extract the sum of the "precipitation_sum" column
    precipitation_sum = api_response_df['precipitation_sum'].sum()

    # Extract the windspeed for the last day (day of incident)
    windspeed = api_response_df.iloc[-1]['windspeed_10m_max']

    # Create a new row with the extracted information
    new_row = pd.DataFrame([[incident['vis_inc_num'], date, mean_temperature, precipitation_sum, windspeed]], columns=master_df.columns)

    # Append the new row to the master DataFrame
    master_df = master_df.append(new_row, ignore_index=True)
    
    print("Done: " + str(index))

# Now master_df contains the extracted information for all incidents, associated with the 'vis_inc_num' column

In [ ]:
# Removing Duplicates from the data
master_df = master_df.drop_duplicates()

# Exporting the weather data to csv file. 
master_df.to_csv("../data/final/weather.csv")